In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyze a codebase with the Vertex AI Gemini 1.5 Pro


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcode%2Fanalyze_codebase_with_gemini_1_5_pro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Eric Dong](https://github.com/gericdong), [Aakash Gouda](https://github.com/aksstar)|

## Overview

Gemini 1.5 Pro introduces a breakthrough long context window of up to 1 million tokens that can help seamlessly analyze, classify and summarize large amounts of content within a given prompt. With its long-context reasoning, Gemini 1.5 Pro can analyze an entire codebase for deeper insights.

In this tutorial, you learn how to analyze an entire codebase with Gemini 1.5 Pro and prompt the model to:

- **Analyze**: Summarize codebases effortlessly.
- **Guide**: Generate clear developer getting-started documentation.
- **Debug**: Uncover critical bugs and provide fixes.
- **Enhance**: Implement new features and improve reliability and security.


## Getting Started

### Install Vertex AI SDK for Python


In [5]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform \
                                        gitpython \
                                        magika

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart runtime (Colab only)

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "gurkomal-playground"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [3]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
)

## Cloning a codebase

You will use repo [Online Boutique](https://github.com/GoogleCloudPlatform/microservices-demo) as an example in this notebook. Online Boutique is a cloud-first microservices demo application. The application is a web-based e-commerce app where users can browse items, add them to the cart, and purchase them. This application consists of 11 microservices across multiple languages.

In [4]:
# The GitHub repository URL
repo_url = "https://github.com/GoogleCloudPlatform/microservices-demo"  # @param {type:"string"}

# The location to clone the repo
repo_dir = "./repo"

#### Define helper functions for processing GitHub repository

In [5]:
import os
import shutil
from pathlib import Path
import requests
import git
import magika

m = magika.Magika()


def clone_repo(repo_url, repo_dir):
    """Clone a GitHub repository."""

    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    os.makedirs(repo_dir)
    git.Repo.clone_from(repo_url, repo_dir)


def extract_code(repo_dir):
    """Create an index, extract content of code/text files."""

    code_index = []
    code_text = ""
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, repo_dir)
            code_index.append(relative_path)

            file_type = m.identify_path(Path(file_path))
            if file_type.output.group in ("text", "code"):
                try:
                    with open(file_path, "r") as f:
                        code_text += f"----- File: {relative_path} -----\n"
                        code_text += f.read()
                        code_text += "\n-------------------------\n"
                except Exception:
                    pass

    return code_index, code_text


def get_github_issue(owner: str, repo: str, issue_number: str) -> str:
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
    }  # Set headers for GitHub API

    # Construct API URL
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"

    try:
        response_git = requests.get(url, headers=headers)
        response_git.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as error:
        print(f"Error fetching issue: {error}")  # Handle potential errors

    issue_data = response_git.json()
    if issue_data:
        return issue_data["body"]
    return ""

#### Create an index and extract content of a codebase

Clone the repo and create an index and extract content of code/text files.

In [6]:
clone_repo(repo_url, repo_dir)

code_index, code_text = extract_code(repo_dir)

## Analyzing the codebase with Gemini 1.5 Pro

With its long-context reasoning, Gemini 1.5 Pro can process the codebase and answer questions about the codebase.

#### Load the Gemini 1.5 Pro model

Learn more about the [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [7]:
MODEL_ID = "gemini-1.5-pro-001"  # @param {type:"string"}

model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "You are a coding expert.",
        "Your mission is to answer all code related questions with given context and instructions.",
    ],
)

#### Define a helper function to generate a prompt to a code related question

In [8]:
def get_code_prompt(question):
    """Generates a prompt to a code related question."""

    prompt = f"""
    Questions: {question}

    Context:
    - The entire codebase is provided below.
    - Here is an index of all of the files in the codebase:
      \n\n{code_index}\n\n.
    - Then each of the files is concatenated together. You will find all of the code you need:
      \n\n{code_text}\n\n

    Answer:
  """

    return prompt

### 1. Summarizing the codebase


Generate a summary of the codebase.

In [9]:
question = """
  Give me a summary of this codebase, and tell me the top 3 things that I can learn from it.
"""

prompt = get_code_prompt(question)
contents = [prompt]

# Generate text using non-streaming method
response = model.generate_content(contents)

# Print generated text and usage metadata
print(f"\nAnswer:\n{response.text}")
print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')
print(f"\nFinish reason:\n{response.candidates[0].finish_reason}")
print(f"\nSafety settings:\n{response.candidates[0].safety_ratings}")


Answer:
```
This codebase is the source code for the "Online Boutique" microservices demo application. It is a cloud-first, web-based e-commerce app that simulates the user journey of browsing products, adding them to a shopping cart, and completing an order. 

The application is built as a set of 11 microservices, each written in a different language (Go, C#, Node.js, Python, and Java) and communicating over gRPC. The services are packaged into Docker containers and deployed to a Kubernetes cluster. Kustomize and Helm are also used to enable easy customization and deployment of the application.

Here are three things you can learn from this codebase:

1. **Microservices architecture:** The app showcases how to build a complex application as a set of loosely coupled, independently deployable microservices. Each service has a well-defined responsibility and communicates with other services over a network. This allows for independent development and scaling of different parts of the app

### 2. Creating a developer getting started guide

Generate a getting started guide for developers. This sample uses the streaming option to generate the content.

In [10]:
question = """
  Provide a getting started guide to onboard new developers to the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```


## Getting Started with the Online Boutique Codebase

Welcome to the Online Boutique

 microservices demo application! This guide will help you get familiar with the codebase

 and start contributing.

**1. Understanding the Architecture**

Online Boutique is a cloud-native microservices demo showcasing various technologies like Kubernetes, gRPC,

 and Google Cloud Platform services. It simulates an e-commerce application where users can browse products, add them to a cart, and purchase them.

The application

...

bash
    skaffold run --default-repo=gcr.io/[PROJECT_ID] # For GKE, replace [PROJECT_ID]
    skaffold run # For local cluster
    ```

**4. Exploring the Code**

- **Microservice Directories:** Each microservice has its

 own directory within the `src` folder.
- **Protocol Buffer Definitions:** The `protos` directory contains the gRPC service definitions using Protocol Buffers.
- **Kubernetes Manifests:** The `kubernetes-manifests` directory contains the Kubernetes deployment files.
- **Kustomize Configurations:** The `k

ustomize` directory provides various configurations for customizing deployments.

**5. Contributing to the Project**

- **Read the Contribution Guidelines:** The [`CONTRIBUTING.md`](.github/CONTRIBUTING.md) file outlines the contribution process and requirements.
- **Submit Pull Requests:** Fork the repository, make

 your changes, and submit a pull request.
- **Join the Discussion:** Participate in discussions on GitHub issues.

We welcome contributions from the community!
```

**Additional Tips:**

- Use an IDE with good support for your chosen programming languages.
- Familiarize yourself with gRPC and Protocol Buffers

.
- Explore the Kustomize components for deploying variations of Online Boutique.
- Use the load generator to test the performance and scalability of the application.

If you have any questions or need further assistance, please don't hesitate to create a GitHub issue. Happy coding!


### 3. Finding bugs

Find the top 3 most severe issues in the codebase.

In [11]:
question = """
  Find the top 3 most severe issues in the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
    def ListRecommendations(self, request, context):
        max

_responses = 5
        # fetch list of products from product catalog stub


        cat_response = product_catalog_stub.ListProducts(demo_pb2.Empty())
        product_ids = [x.id

 for x in cat_response.products]
        filtered_products = list(set(product_ids)-set(request.product_ids))


...

(p.catalog.Products) == 0 {
    err := loadCatalog(&p.catalog)
    if err != nil {
      return []*pb.Product{}
    }
  }
```

**Issue 2:** `parseCatalog` in `productcatalogservice/product_catalog

.go` reloads the catalog unnecessarily on every request when `reloadCatalog` is enabled. This results in a significant delay in processing requests.

**Proposed Solution:** Implement a more efficient mechanism for catalog reloading, such as using a file watcher or a timer to trigger reloading only when the `products.json` file

 changes. This would minimize unnecessary reloads and improve response times.

```python
    def getSupportedCurrencies(self, request, context):
        # MISSING: implementation to call Currency Service
        return demo_pb2.GetSupportedCurrenciesResponse(
            currency_codes=[
                "USD", "

EUR", "CAD", "GBP", "JPY"
            ]
        )
```

**Issue 3:** `getSupportedCurrencies` in `currencyservice/server.js` returns a static list of currencies. This should call the `CurrencyService` to fetch supported currencies dynamically.

**

Proposed Solution:** Implement the call to the `CurrencyService` within `getSupportedCurrencies` to retrieve the list of supported currencies. This would ensure the returned list is up-to-date and aligned with the currencies supported by the `CurrencyService`. 


### 4. Fixing bug

Find the most severe issue in the codebase that can be fixed and provide a code fix for it.


In [12]:
question = """
  Find the most severe bug in the codebase that you can provide a code fix for.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

diff
--- a/src/cartservice/src/cartstore/Alloy

DBCartStore.cs
+++ b/src/cartservice/src/

cartstore/AlloyDBCartStore.cs
@@ -62,7 +62,7 @@
                 }
                 var totalQuantity = quantity

 + currentQuantity;
 
-                var insertCmd = $"INSERT INTO {tableName} (userId, productId, quantity) VALUES ('{userId}', '{

productId}', {totalQuantity})";
+                var insertCmd = $"INSERT INTO {tableName} (userId, productId, quantity) VALUES ('{userId}', '{productId}', {totalQuantity}) ON CONFLICT (userId, productId) DO

 UPDATE SET quantity = EXCLUDED.quantity";
                 await using (var cmdInsert = dataSource.CreateCommand(insertCmd))
                 {
                     await Task.Run(() =>

```

The bug is in the `AddItem

Async` method of `AlloyDBCartStore.cs`. The insert command does not account for conflicts in `userId` and `productId`, and will throw an exception if an insert is attempted for a `userId` and `productId` that already exists.

The fix is to update the `insertCmd` to use

 `ON CONFLICT` to handle the case of an existing `userId` and `productId`, updating the `quantity` to the `EXCLUDED.quantity` in that case:

```sql
INSERT INTO {tableName} (userId, productId, quantity) VALUES ('{userId}', '{productId}', {totalQuantity

}) ON CONFLICT (userId, productId) DO UPDATE SET quantity = EXCLUDED.quantity
```

This will prevent the exception from being thrown and properly update the existing quantity if a `userId`/`productId` pair already exists.


### 5. Implementing a feature request using Function Calling

Generate code to implement a feature request.

Get feature request text from GitHub Issue

In [13]:
# Function declaration with detailed docstring
extract_details_from_url_func = FunctionDeclaration(
    name="extract_details_from_url",
    description="Extracts owner, repository name, and issue number details from a GitHub issue URL",
    parameters={
        "type": "object",
        "properties": {
            "owner": {
                "type": "string",
                "description": "The owner of the GitHub repository.",
            },
            "repo": {
                "type": "string",
                "description": "The name of the GitHub repository.",
            },
            "issue_number": {
                "type": "string",
                "description": "The issue number to fetch the body of.",
            },
        },
    },
)

# Tool definition
extraction_tool = Tool(function_declarations=[extract_details_from_url_func])

FEATURE_REQUEST_URL = (
    "https://github.com/GoogleCloudPlatform/microservices-demo/issues/2205"
)

# Prompt content
prompt_content = f"What is the feature request of the following {FEATURE_REQUEST_URL}"

# Model generation with tool usage
response = model.generate_content(
    [prompt_content],
    generation_config=GenerationConfig(temperature=0),
    tools=[extraction_tool],
)
# Extract parameters from model response
function_call = response.candidates[0].function_calls[0]

# Fetch issue details from GitHub API if function call matches
if function_call.name == "extract_details_from_url":
    issue_body = get_github_issue(
        function_call.args["owner"],
        function_call.args["repo"],
        function_call.args["issue_number"],
    )

IPython.display.Markdown(f"Feature Request:\n{issue_body}")

Feature Request:
### Describe request or inquiry 
helm chart frontend-external support config service type nodeport, like this
```
helm install xxx --set frontend.service.type=NodePort
```

### What purpose/environment will this feature serve? 

This feature enables quick access to the front-end web interface of microservices without the need for additional configuration work.

I hope to quickly access the web interface after deploying this microservice in the local environment, without the need for additional loadbalancer or ingress configuration, just nodeport is enouth.

But now that the deployment is complete, I must manually edit the service and modify the nodeport. If Helm provides parameters to set the nodeport, I don't need to。


Use the GitHub Issue text to implement the feature request

In [14]:
# Combine feature request with URL and get code prompt
question = (
    "Implement the following feature request" + FEATURE_REQUEST_URL + "\n" + issue_body
)

prompt = get_code_prompt(question)

# Generate code response
response = model.generate_content([prompt])
IPython.display.Markdown(response.text)  # Display in Markdown format

```diff
--- a/helm-chart/templates/frontend.yaml
+++ b/helm-chart/templates/frontend.yaml
@@ -32,6 +32,8 @@
   cymbalBranding: false
   # One of: local, gcp, aws, azure, onprem, alibaba. When not set, defaults to "local" unless running in GKE, otherwise auto-sets to gcp.
   platform: local
+  # Type of service, could be LoadBalancer, NodePort or ClusterIP.
+  serviceType: LoadBalancer
   singleSharedSession: false
   virtualService:
     create: false
@@ -86,7 +88,7 @@
   name: {{ .Values.frontend.name }}
   namespace: {{ .Release.Namespace }}
   labels:
-    app: {{ .Values.frontend.name }}
+    app: {{ include "common.name" . }}
 spec:
   type: ClusterIP
   selector:
@@ -99,7 +101,8 @@
 metadata:
   name: {{ .Values.frontend.name }}-external
   namespace: {{ .Release.Namespace }}
-spec:
+  labels:
+    app: {{ include "common.name" . }}
   type: {{ .Values.frontend.serviceType }}
   selector:
     app: {{ .Values.frontend.name }}

```

### 6. Creating a troubleshooting guide

Create a troubleshooting guide to help resolve common issues.

In [15]:
question = """
    Provide a troubleshooting guide to help resolve common issues.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```


## Online Boutique Troubleshooting Guide

This guide helps you troubleshoot common issues encountered when

 running the Online Boutique microservices demo.

**Table of Contents**

-

 [General Troubleshooting](#general-troubleshooting)
  - [Check Pod Status](#check-pod-status)
  - [Examine Pod Logs](#examine

-pod-logs)
  - [Verify Service Connectivity](#verify-service-connectivity)
  - [Resource Constraints](#resource-constraints)
-

...

 and logs.
    - Verify `AD_SERVICE_ADDR` environment variable in the frontend deployment.

## Troubleshooting Tools

### kubectl

- `kubectl get pods`: List all pods and their status.
- `kubectl describe pod <pod-name>`: Get detailed information about a pod, including events

 and status conditions.
- `kubectl logs <pod-name>`: View logs for a specific pod.
- `kubectl exec -it <pod-name> bash`: Execute commands inside a pod.

### skaffold

- `skaffold dev`: Build and deploy your application continuously, watching for code changes

.
- `skaffold debug`: Debug your application running in Kubernetes.
- `skaffold run`: Build, push, and deploy your application once.

### Docker

- `docker ps`: List running containers.
- `docker logs <container-name>`: View logs for a specific container.


- `docker exec -it <container-name> bash`: Execute commands inside a container.

### Google Cloud Operations Suite

- **Logging**: View and analyze logs from your Online Boutique services.
- **Monitoring**: Monitor metrics and set up alerts for your Online Boutique services.
- **Tracing**: Trace requests

 across your Online Boutique microservices.
- **Profiler**: Identify performance bottlenecks in your Online Boutique services.

This guide provides a starting point for troubleshooting Online Boutique. If you encounter issues not covered here, refer to the official documentation for the respective services and tools.
```

### 7. Making the app more reliable

Recommend best practices to make the application more reliable.


In [16]:
question = """
  How can I make this application more reliable? Consider best practices from https://www.r9y.dev/
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import os
import random
import time
import traceback
from concurrent

 import futures

import googlecloudprofiler
from google.auth.exceptions import Default

CredentialsError
import grpc

import demo_pb2
import demo_pb2_grpc
from grpc_health.v1 import health_pb2


from grpc_health.v1 import health_pb2_grpc

from opentelemetry import trace
from opentelemetry.instrumentation

...

 os.environ.get('PORT', "8080")
    catalog_addr = os.environ.get('PRODUCT_CATALOG_SERVICE_ADDR', '')
    if catalog_addr == "":
        raise Exception('PRODUCT_CATALOG_SERVICE_ADDR environment variable not set')
    logger.

info("product catalog address: " + catalog_addr)
    with grpc.insecure_channel(catalog_addr) as channel:
      product_catalog_stub = demo_pb2_grpc.ProductCatalogServiceStub(channel)
      # create gRPC server
      server = grpc.

server(futures.ThreadPoolExecutor(max_workers=10))

      # add class to gRPC server
      service = RecommendationService()
      demo_pb2_grpc.add_RecommendationServiceServicer_to_server(service, server)
      health_pb2_grpc.add

_HealthServicer_to_server(service, server)

      # start server
      logger.info("listening on port: " + port)
      server.add_insecure_port('[::]:'+port)
      server.start()

      # keep alive
      try:


          while True:
              time.sleep(10000)
      except KeyboardInterrupt:
              server.stop(0)
```

### 8. Making the app more secure

Recommend best practices to make the application more secure.

In [18]:
question = """
  How can you secure the application?
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

This

 application is a demo application and includes a number of insecure defaults for ease of deployment

.  Production deployments should consider the following:

* **Enable Authentication and Authorization

** The current application does not require any authentication or authorization.  Consider adding user authentication and role-based access control (RBAC) to protect sensitive data and

 operations.  
* **HTTPS/TLS**: The application is currently served over HTTP. Enable HTTPS for all services and use TLS certificates to encrypt data in transit

.
* **Secure Sensitive Data:** The application uses hardcoded credentials and API keys.  Store sensitive data, like API keys and database passwords, in a secure location like [Secret Manager](https://cloud.google.com/secret

-manager) or [Hashicorp Vault](https://www.vaultproject.io/).
* **Enable Secure Defaults**: Configure more secure defaults for Kubernetes deployments, including:
    * **Resource Limits**: Set resource requests and limits

 for all containers to prevent resource starvation attacks.
    * **Pod Security Policies**: Use [Pod Security Standards](https://kubernetes.io/docs/concepts/security/pod-security-standards/) to enforce container and pod level security policies.  This can help restrict access to host resources, prevent privilege escalation,

 and enforce container immutability.
    * **Network Policies**: Use network policies to restrict traffic flow between services. This can help prevent unauthorized access to sensitive services. 
* **Regular Security Scans:** Use vulnerability scanners to check for vulnerabilities in container images and dependencies.
* **Defense in Depth:** Use a

 combination of security measures to protect your application. This includes using firewalls, intrusion detection systems, and security logging and monitoring.



### 9. Learning the codebase

Create a quiz about the concepts used in the codebase.

In [19]:
question = """
  Create a quiz about the concepts used in my codebase to help me solidify my understanding.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import random

questions = [
    {
        "question":

 "What is the purpose of the `skaffold.yaml` file?",


        "options": [
            "To define the infrastructure for deploying the application.",
            "To define the build and deployment process for the application.",


            "To define the unit tests for the application.",
            "To define the network policies for the application."
        ],
        "answer": "

...


        except ValueError:
            print("Invalid input. Please enter a number between 1 and 4.")
    
    if q['options'][user_answer-1] == q['answer']:
        print("Correct!")
        score += 1
    else:
        print(f

"Incorrect. The answer is: {q['answer']}")

print(f"\nYour final score: {score}/{len(questions)}")
```

This quiz covers several key aspects of your provided codebase:

* **Build and Deployment:** Questions about `skaffold.yaml`, `kustom

ize` components, and `.github/workflows` probe the user's understanding of how the application is built, configured, and deployed.
* **Microservice Architecture:** Questions about inter-service communication protocols (gRPC) and the role of specific services (e.g., `loadgenerator`, `frontend

`) test comprehension of the overall system design.
* **Environment and Configuration:**  Questions related to environment variables like `CYMBAL_BRANDING`, `EXTRA_LATENCY`, and database choices highlight how the application's behavior can be customized. 
* **Technology Choices:** The quiz assesses knowledge of languages

 used (Go, Node.js) and databases (Redis, Spanner).

By taking this quiz, you'll be able to reinforce your knowledge of these fundamental concepts within your codebase.  


### 10. Creating a quickstart tutorial

Create an end-to-end quickstart tutorial for a specific component.


In [20]:
question = """
  Please write an end-to-end quickstart tutorial that introduces AlloyDB,
  shows how to configure it with the CartService,
  and highlights key capabilities of AlloyDB in context of the Online Boutique application.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```


## Online Boutique - Quickstart with AlloyDB

This quickstart guide outlines

 how to configure the Online Boutique application's CartService to utilize AlloyDB as

 its persistent data store. It highlights AlloyDB's key capabilities and advantages in this specific context.

### What is AlloyDB?

AlloyDB is a

 fully managed, PostgreSQL-compatible database service on Google Cloud. It is designed for high performance transactional workloads, offering superior performance, scalability, and availability compared to standard

...

 with Kustomize:**

   * From the root of the repository, navigate to the `kustomize` directory.
   * Add the `alloydb` component to your `kustomization.yaml`
     ```yaml
     apiVersion: kustomize.config.k8s.io/

v1beta1
     kind: Kustomization
     resources:
     - base
     components:
     - components/alloydb
     ```
   * Apply the configuration using:
     ```bash
     kubectl apply -k .
     ```

### Key Capabilities in Online Boutique

*

 **Real-time Cart Updates:** AlloyDB's high-performance processing ensures that cart additions, removals, and updates are reflected immediately, providing a responsive and accurate shopping experience.
* **Consistent View of Cart Data:**  AlloyDB's strong consistency guarantees that users always see the most up-to-date

 cart contents, regardless of concurrent access.
* **Reliable Checkout Process:**  AlloyDB's robust transaction handling ensures that order placement and payment processing are reliable and atomic, minimizing errors and data inconsistencies.

### Conclusion

By following these steps, you successfully configured Online Boutique's CartService to use AlloyDB,

 leveraging its strengths for a high-performance and reliable e-commerce experience.
```

This tutorial offers a practical and easy-to-follow guide for integrating AlloyDB with the Online Boutique application, showcasing its advantages for improving the cart service's performance and reliability. 


### 11. Creating a Git Changelog Generator

Understanding changes made between Git commits and highlighting the most important aspects of the changes.

In [21]:
### Fetches commit IDs from a local Git repository on a specified branch.

repo = git.Repo(repo_dir)
branch_name = "main"
commit_ids = [
    commit.hexsha for commit in repo.iter_commits(branch_name)
]  # A list of commit IDs (SHA-1 hashes) in reverse chronological order (newest first)

if len(commit_ids) >= 2:
    diff_text = repo.git.diff(commit_ids[0], commit_ids[1])

    question = """
      Given the above git diff output, Summarize the important changes made.
    """

    prompt = diff_text + question + code_text
    contents = [prompt]

    responses = model.generate_content(contents, stream=True)
    for response in responses:
        IPython.display.Markdown(response.text)

The

 diff output shows a significant reduction in the size of the `node_modules`

 directory in the `src/currencyservice` directory. This is likely

 due to the removal of unnecessary dependencies and a downgrade of some dependencies to older versions. The specific changes include:

* **Removal of:**
    * `

@babel/helper-string-parser`
    * `@babel/helper-validator-identifier`
    * `@babel/types`


...

.1.0 to 12.3.2
    * `markdown-it-anchor` from 8.6.7 to 8.6.5
    * `marked` from 4.3.0 to 4.2.3
    * `mdurl` from

 2.0.0 to 1.0.1
    * `minimist` from 1.2.8 to 1.2.7
    * `protobufjs` from 7.2.4 to 7.1.2
    * `protobufjs-cli`

 from 1.1.1 to 1.0.2
    * `tmp` from 0.2.3 to 0.2.1
    * `uc.micro` from 2.1.0 to 1.0.6
    * `uglify-js`

 from 3.19.2 to 3.17.4
    * `underscore` from 1.13.7 to 1.13.6
    * `word-wrap` from 1.2.5 to 1.2.4

These changes likely

 result in a smaller and more efficient `currencyservice` application. 


## Conclusion

In this tutorial, you've learned how to use the Gemini 1.5 Pro to analyze a codebase and prompt the model to:

- Summarize codebases effortlessly.
- Generate clear developer getting-started documentation.
- Uncover critical bugs and provide fixes.
- Implement new features and improve reliability and security.
- Understanding changes made between Git commits